In [1]:
import pandas as pd
import pymongo
from dotenv import load_dotenv
import os

In [2]:
load_dotenv()
ds = pd.read_excel("patients_synthetic.xlsx") #COLOCAR DATASET

In [3]:
client = pymongo.MongoClient(os.getenv("URI"))

In [12]:
db = client["Nefro"]
collection = db["datasets"]
collection.delete_many({"name": "patients_synthetic"})
collection = db["exams"]
collection.delete_many({"dataset": "patients_synthetic"})
collection = db["pacients"]
collection.delete_many({"dataset": "patients_synthetic"})


In [7]:
codigos = ds['Código'].unique().tolist()

In [8]:

def import_to_db(obj, exams):
    db = client["Nefro"]
    collection = db["pacients"]
    # Insert the dataset into the database
    collection.update_one(
    {"codigo": obj["codigo"], "dataset":  obj["dataset"]},
    {"$set": obj},
    upsert=True
    )
    collection = db["exams"]
    collection.update_one(
    {"codigo": exams["codigo"], "dataset":  exams["dataset"]},
    {"$set": exams},
    upsert=True
    )
    print("imported")

def import_ds(ds):
    db = client["Nefro"]
    collection = db["datasets"]
    # Insert the dataset into the database
    collection.update_one(
    {"name": ds["name"]},
    {"$set": ds},
    upsert=True
    )
    print("Dataset registred")


In [9]:
def dataset_info(dataset):
    first_month = dataset['Mês/ano'].min()
    last_month = dataset['Mês/ano'].max()
    pacients = dataset.drop_duplicates(subset=['Código'], keep='last')

    total_pacients = pacients.count()["Código"]
    desfecho_counts = pacients['Desfecho'].value_counts()
    vivos = desfecho_counts.get('Vivo', 0)
    mortos = desfecho_counts.get('Óbito', 0)

    return vivos, mortos,total_pacients, first_month, last_month

def get_ds_exams_info(dataset):
    exams = dataset.drop(columns=['Unnamed: 0', 'Código', 'Mês/ano', 'Sexo', 'Idade', 'Desfecho'])
    exams_summary = exams.describe().T

    head = ["Exame", "25%", "50%", "75%", "Max", "Média", "Min", "Desvio Padrão"]
    body = [
        {
            "Exame": index,
            "Média": float(row["mean"]),
            "Desvio Padrão": float(row["std"]),
            "Min": float(row["min"]),
            "25%": float(row["25%"]),
            "50%": float(row["50%"]),
            "75%": float(row["75%"]),
            "Max": float(row["max"])
        }
        for index, row in exams_summary.iterrows()
    ]

    return {
        "head": head,
        "body": body
    }

def get_dead_Alive(dataset):
    pacients = dataset.drop_duplicates(subset=['Código'], keep='last')
    desfecho_counts = pacients['Desfecho'].value_counts()
    vivos = desfecho_counts.get('Vivo', 0)
    mortos = desfecho_counts.get('Óbito', 0)

    return [
        {"id": 0, "label": "mortos", "value": int(mortos)},
        {"id": 1, "label": "vivos", "value": int(vivos)}
        ]

In [10]:
def add_pacient(codigo,dataset_name):
    pacient = ds[ds['Código'] == codigo]
    pacient = pacient.drop(columns=['Unnamed: 0'])
    pacient = pacient.sort_values(by=['Mês/ano'])
    pacient["Mês/ano"] = pd.to_datetime(pacient["Mês/ano"])


    transformed_data = []
    for column in pacient.columns:
        if column != "Mês/ano" and column != "Código" and column != "Sexo" and column != "Idade" and column != "Desfecho":
            if column == "Anti-HBs" or column == "HBs Ag" or column == "Anti-HIV":
                pd.set_option('future.no_silent_downcasting', True)
                pacient[column] = pacient[column].replace('Positivo', 1)
                pacient[column] = pacient[column].replace('Negativo', 0)
                
            data_points = [
                {"x": date, "y": value}
                for date, value in zip(pacient["Mês/ano"], pacient[column])
            ]
            transformed_data.append({
                "name": column,
                "data": data_points
            })


    pacient['Mês/ano'] = pd.to_datetime(pacient['Mês/ano'])
    first_month = pacient['Mês/ano'].min()
    last_month = pacient['Mês/ano'].max()
    months_of_treatment = (last_month.year - first_month.year) * 12 + last_month.month - first_month.month + 1
    pacient = pacient.drop_duplicates(subset=['Código'], keep='last')


    person = {
        "codigo": str(pacient['Código'].values[0]),
        "sexo": pacient['Sexo'].values[0],
        "idade": int(pacient['Idade'].values[0]),
        "desfecho": pacient['Desfecho'].values[0],
        "inicio": first_month,
        "fim": last_month,
        "range": months_of_treatment,
        "dataset": dataset_name
    }

    exams ={
        "codigo":str(pacient['Código'].values[0]),
        "dataset": dataset_name,
        "exams": transformed_data
    }
    import_to_db(person, exams)


def add_dataset(dataset,dataset_name):
    vivos, mortos ,total_pacients, inicio,fim = dataset_info(dataset)
    codigos = dataset['Código'].unique().tolist()
    to_import = {
        "name": dataset_name,
        "pacientes": int(total_pacients),
        "vivos": int(vivos),
        "mortos": int(mortos),
        "inicio": inicio,
        "fim": fim,
        "estatisticas": get_ds_exams_info(dataset),

    }
    import_ds(to_import)
    for codigo in codigos:
        add_pacient(codigo,dataset_name)

In [11]:
add_dataset(ds,"patients_synthetic")

Dataset registred
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
imported
i